In [1]:
# Import necessary packages
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import pickle
import faiss

# Import LangChain and OpenAI components
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Import our custom retriever classes
from src.rag.retriever import CustomRetriever, RerankingRetriever

# Load environment variables from .env file
load_dotenv()

# Set the working directory to the project root if running from the 'notebooks' folder
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    
# Load data and models from paths specified in environment variables
df = pd.read_parquet(os.getenv("PARQUET_PATH"))
faiss_index = faiss.read_index(os.getenv("FAISS_INDEX_PATH"))
with open(os.getenv("ID_MAPPING_PATH"), "rb") as f:
    id_mapping = pickle.load(f)

# Initialize the OpenAI embedding model
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
# 1. Initialize the base retriever
# We retrieve more documents (k=20) initially to provide a good selection for the reranker to work with.
custom_retriever = CustomRetriever(
    embeddings=embeddings,
    faiss_index=faiss_index,
    id_mapping=id_mapping,
    documents_df=df,
    k=20
)

# 2. Load and create the reranker prompt
with open("src/prompts/reranker_prompt.txt", "r", encoding="utf-8") as f:
    reranker_prompt_template = f.read()

reranker_prompt = PromptTemplate.from_template(reranker_prompt_template)

# 3. Initialize the reranker LLM
# We use an efficient yet cost-effective model (gpt-4o-mini).
# A temperature of 0 ensures deterministic, fact-based operation.
reranker_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# 4. Initialize the RerankingRetriever
# This component ties the process together. The "k" parameter here 
# specifies the final number of documents to be returned to the user.
reranking_retriever = RerankingRetriever(
    retriever=custom_retriever,
    llm=reranker_llm,
    reranker_prompt=reranker_prompt,
    embeddings=embeddings,  # Pass the embeddings model for snippet analysis
    k=5
)

In [3]:
# A specific query asking for a legal definition in Hungarian
query = "Milyen eséllyel ítél a bíróság az alperes részére, ha az egy civil szervezet ?"

# Retrieve relevant documents using the RerankingRetriever's invoke method
final_docs = reranking_retriever.invoke(query)

# Print the results
print(f"Query: {query}\\n")
print("--- Reranked Documents ---\\n")

for i, doc in enumerate(final_docs, 1):
    print(f"📄 Document {i} (ID: {doc.metadata.get('doc_id')})")
    print(f"   Reranker Score: {doc.metadata.get('reranker_score', 'N/A')}")
    print(f"   Reason: {doc.metadata.get('reranker_reason', 'N/A')}")
    
    # Display the full, multi-part snippet that was used for reranking
    best_snippet = doc.metadata.get('best_snippet_for_reranker', 'Snippet not found.')
    print(f"\n   --- Snippet Sent to Reranker ---\n{best_snippet}")
    print(f"\n   --- Original Document (First 500 chars) ---\n{doc.page_content[:500]}...")
    print("\n" + "="*80 + "\n")


Query: Milyen eséllyel ítél a bíróság az alperes részére, ha az egy civil szervezet ?\n
--- Reranked Documents ---\n
📄 Document 1 (ID: Kfv.37173/2007/7)
   Reranker Score: 5
   Reason: A dokumentum részletesen tárgyalja a civil szervezetek jogi helyzetét és a bíróság döntéseit az ilyen ügyekben.

   --- Snippet Sent to Reranker ---
továbbá a Magyar Államnak - felhívásra - 36.000 (harminchatezer) forint felülvizsgálati eljárási illetéket. Ez ellen az ítélet ellen további felülvizsgálatnak nincs helye. I n d o k o l á s : A felperes egyebek mellett egy 1996-óta fokozottan védetté nyilvánított területen gazdálkodik. A gazdálkodásra vonatkozó előírásokat egy 1998. körzeti erdőterv tartalmazza azzal, hogy a felperesnek 2008-ig e szerint kell eljárni. A területre a felperes 2001-ben az engedélyezetten felül további 6 ha erdőrészletre

---

ezt meghaladóan 3x2 ha terület tarvágásra kért engedélyhez nem járult hozzá, a felperest ezen tiltás következtében a következő években 20 ha erdő értékes 